In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from itertools import permutations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

print("Imports are ok!")

Imports are ok!


The Data:
The crash data I am using is from the Colorado DOT and can be downloaded here: https://www.codot.gov/safety/traffic-safety/data-analysis/crash-data

I specifically am using the 2022 data because it was the last year that alcohol and drug use was included in the dataset and I suspect that will be a large factor in whether or not the crash results in a serious injury.

In [16]:
df = pd.read_excel("CDOTRM_CD_Crash_Listing_-_2022.xlsx", na_filter=False)

print(df.columns)
print("Data is ok!")

Index(['CUID', 'System Code', 'Rd_Number', 'Rd_Section', 'City_Street',
       'Crash Date', 'Crash Time', 'Agency Id', 'City', 'County', 'Latitude',
       'Longitude', 'Location 1', 'Link', 'Location 2', 'Location',
       'Road Description', 'First HE', 'Second HE', 'Third HE', 'Fourth HE',
       'MHE', 'Crash Type', 'Wild Animal', 'Number Killed', 'Number Injured',
       'Injury 00', 'Injury 01', 'Injury 02', 'Injury 03', 'Injury 04',
       'Total Vehicles', 'Secondary Crash', 'Construction Zone', 'School Zone',
       'Road Contour Curves', 'Road Contour Grade', 'Road Condition',
       'Lighting Conditions', 'Weather Condition', 'Lane Position',
       'TU-1 Direction', 'TU-2 Direction', 'TU-1 Movement', 'TU-2 Movement',
       'TU-1 Type', 'TU-2 Type', 'TU-1 Hit And Run', 'TU-2 Hit And Run',
       'TU-1 Driver Action', 'TU-2 Driver Action',
       'TU-1 Human Contributing Factor', 'TU-2 Human Contributing Factor',
       'TU-1 Age', 'TU-2 Age', 'TU-1 Sex ', 'TU-2 Sex',
     

Get data ready to be used by an unsupervised algorithm. I plan on using a NMF model because I think it will do a good job handling the large number of features that the crash data contains.

In [19]:
#make a df from just the columns we want that have each been bagged 
#we will create multiple bags of the columns that I suspect will be telling for the survivability of the accident and then combine them into one dratframe for the NMF model
#convert the words to numerical values using the bag of words concept and a vectorizer from sklearn (code for this taken from my homwork 4)
locationVect = CountVectorizer()
locBag = locationVect.fit_transform(df.Location)
locFeatures = locationVect.get_feature_names_out()
locDf = pd.DataFrame(locBag.toarray(), columns=locFeatures)
print(locDf.head())
print(len(locDf.index))


   center  crossed  intersection  into  island  lanes  left  median  off  on  \
0       0        0             0     0       0      0     0       0    0   1   
1       0        0             0     0       0      0     0       0    1   0   
2       1        0             0     0       1      0     0       1    0   0   
3       1        0             0     0       1      0     0       1    0   0   
4       0        0             0     0       0      0     0       0    0   1   

   opposing  private  property  ran  right  roadway  side  vehicle  
0         0        0         0    0      0        1     0        0  
1         0        0         0    1      1        0     1        0  
2         0        0         0    0      0        0     0        0  
3         0        0         0    0      0        0     0        0  
4         0        0         0    0      0        1     0        0  
95363


In [20]:
condVect = CountVectorizer()
condBag = condVect.fit_transform(df["Road Condition"])
condFeatures = condVect.get_feature_names_out()
condDf = pd.DataFrame(condBag.toarray(), columns=condFeatures)
print(condDf.head())
print(len(condDf.index))

   dry  foreign  gravel  icy  material  milled  muddy  road  roto  sand  \
0    0        0       0    0         0       0      0     0     0     0   
1    0        0       0    0         0       0      0     0     0     0   
2    1        0       0    0         0       0      0     0     0     0   
3    0        0       0    0         0       0      0     0     0     0   
4    0        0       0    0         0       0      0     0     0     0   

   slushy  snowy  treatment  visible  wet  
0       0      1          0        0    0  
1       0      1          0        0    0  
2       0      0          0        0    0  
3       0      1          0        0    0  
4       0      1          0        0    0  
95363


In [21]:
carVect = CountVectorizer()
carBag = carVect.fit_transform(df["TU-1 Type"])
carFeatures = carVect.get_feature_names_out()
carDf = pd.DataFrame(carBag.toarray(), columns=carFeatures)
print(carDf.head())
print(len(carDf.index))

   001  10  16  all  atv  autocycle  bus  buses  car  commerce  ...  transit  \
0    0   0   0    0    0          0    0      0    0         0  ...        0   
1    0   0   0    0    0          0    0      0    0         0  ...        0   
2    0   0   0    0    0          0    0      0    0         0  ...        0   
3    0   0   0    0    0          0    0      0    0         0  ...        0   
4    0   0   0    0    0          0    0      0    1         0  ...        0   

   truck  trucks  type  unknown  utility  van  vehicle  with  working  
0      0       0     0        1        0    0        0     0        0  
1      0       0     0        0        0    0        0     0        0  
2      0       0     0        1        0    0        0     0        0  
3      0       0     0        1        0    0        0     0        0  
4      0       0     0        0        0    1        0     0        0  

[5 rows x 53 columns]
95363


In [22]:
alcVect = CountVectorizer()
alcBag = alcVect.fit_transform(df["TU-1 Alcohol Suspected"])
alcFeatures = alcVect.get_feature_names_out()
alcDf = pd.DataFrame(alcBag.toarray(), columns=alcFeatures)
print(alcDf.head())
print(len(alcDf.index))

   breath  method  no  observed  other  preliminary  sfst  test  unknown  yes
0       0       0   0         0      0            0     0     0        0    0
1       0       1   0         0      1            0     0     0        0    1
2       0       0   0         0      0            0     0     0        0    0
3       0       0   0         0      0            0     0     0        0    0
4       0       0   1         1      0            0     0     0        0    0
95363


In [18]:
marVect = CountVectorizer()
marBag = marVect.fit_transform(df["TU-1 Marijuana Suspected"])
marFeatures = marVect.get_feature_names_out()
marDf = pd.DataFrame(marBag.toarray(), columns=marFeatures)
print(marDf.head())
print(len(marDf.index))

   marijuana  not  suspected  unknown
0          0    0          0        0
1          1    1          1        0
2          0    0          0        0
3          0    0          0        0
4          1    1          1        0
95363


In [25]:
#combine all bags into one 
bag = pd.concat([locDf, condDf, carDf, alcDf, marDf], axis=1)

#create train test split
Xlen = int(len(bag.index) * .8)

X = bag[0:Xlen].copy()
y = bag[Xlen:].copy()

print("Train Length: ",len(X.index))
print("Test Length: ",len(y.index))

Train Length:  76290
Test Length:  19073


Machine Learning Question: Predict whether or not a car crash will have serious injuries 

This prediction could allow for triage of calls in busy cities in circumstances where calls outnumber the available EMTs that can respond

For this data set we will consider serious injuries to be values in the Injury03 (suspected major injury) and the Injury04 (suspected fatality) fields.

In [26]:
#train the model
#2 components because we are trying to predict lethal and not lethal crashes
model_nmf = NMF(n_components=2, init='nndsvd', beta_loss='frobenius', solver='cd', max_iter=200).fit(X)

In [33]:
#get predictions from the model for the test and training set 
y_pred_raw = model_nmf.transform(y)
X_pred_raw = model_nmf.transform(X)

#combine the arrays into actual predictions about the category
y_pred = np.argmax(y_pred_raw, axis=1)
X_pred = np.argmax(X_pred_raw, axis=1)

print(y_pred)
print(X_pred)


[0 0 1 ... 1 0 0]
[0 0 0 ... 1 0 1]


In [31]:
#filter to only accidents where there was an injury
def MajorInjury(row):
    if(int(row["Injury 03"]) >= 1 or int(row["Injury 04"]) >= 1):
        return(1)
    else:
        return(0)
df["MajorInjury"] = df.apply(MajorInjury, axis=1)

print(df["MajorInjury"].head())
print(len(df["MajorInjury"].index))

0    0
1    0
2    0
3    0
4    0
Name: MajorInjury, dtype: int64
95363


In [38]:
#test accuracy of the predictions
#for this data set we want to tell the difference

majorInjury = df["MajorInjury"].tolist()

#test against the training set and test set 
#keep track of pos0 and pos1 correct values because we have to test both permutations of the predictions
trainPos0 = 0
trainPos1 = 0
testPos0 = 0
testPos1 = 0

#this is just to keep the indexing right as we measure accuracy
y_pred = X_pred.tolist() + y_pred.tolist()

for i in range(len(df.index)):
    if(i < Xlen):
        if(majorInjury[i] == X_pred[i]):
            trainPos0 += 1
        elif(majorInjury[i] != X_pred[i]):
            trainPos1 += 1
    else:
        if(majorInjury[i] == y_pred[i]):
            testPos0 += 1
        elif(majorInjury[i] != y_pred[i]):
            testPos1 += 1


print("For the training set")
if(trainPos0 > trainPos1):
    print("Pos0 had higher accuracy")
    print("Accuracy: ",trainPos0/Xlen)
else:
    print("Pos1 had higher accuracy")
    print("Accuracy: ",trainPos1/Xlen)


print("For the test set")
if(trainPos0 > trainPos1):
    print("Pos0 had higher accuracy")
    print("Accuracy: ",testPos0/((len(df.index)-Xlen)))
else:
    print("Pos1 had higher accuracy")
    print("Accuracy: ",testPos1/((len(df.index)-Xlen)))
    

For the training set
Pos0 had higher accuracy
Accuracy:  0.5721981911128589
For the test set
Pos0 had higher accuracy
Accuracy:  0.5798248833429456


Conclusion:

The NMF model can predict whether a given crash will have a serious injury with about a 58% accuracy based off of the location of the crash, the road condition, the car being driven, and whether the driver was under the influence of aclohol or marajuana. 

I did expect it to be better at predicting but I guess the factors that I chose were not as impactful on their own as I previously though they would be. Going forward I think trying different models and perhaps using a combination of supervised and unsupervised methods may lead to better results.